In [0]:
import os
#SCOPE = 'silo-key-vault-scope'
STORAGE_ACCOUNT_NAME = 'smlake'
CONTAINER_NAME = 'bronze'
MOUNT_POINT = os.path.join('/mnt', STORAGE_ACCOUNT_NAME, CONTAINER_NAME)
# Mount Azure blob
try:
    #key = dbutils.secrets.get(scope=SCOPE, key=SMLAKE-KEY)
    
    configs = {
        'source': f'wasbs://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.blob.core.windows.net',
        'mount_point': MOUNT_POINT,
        'extra_configs': {f'fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.blob.core.windows.net': 'sHyeld1YUP5i3Tixfn8s41SibTcZ+DsN35tfsK7noQXDGGjHk0frX5DGuUaFuomachwvv8OY5bVL6NOWqdW9EQ=='}    
    }
    if any(mount.mountPoint == MOUNT_POINT for mount in dbutils.fs.mounts()):
        print("Already Mounted")
    else:
        dbutils.fs.mount(**configs)
except Exception as error:
    messages = str(error).split('\n')[:2]
    message = '\n'.join(messages)
    logger.warning(message)

Already Mounted

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
#spark.conf.set('spark.sql.adaptive.enabled', 'true')

#paths
ZoomDataLoad = MOUNT_POINT + "/ZoomData/formattedAccountCallLogs.parquet"
rm_DVStatementRequestActivityRecords = MOUNT_POINT + "/OnBase/rm_DVStatementRequestActivityRecords.parquet"
rm_DVStatementRequests = MOUNT_POINT + "/OnBase/rm_DVStatementRequests.parquet"
rm_DVVendorContacts = MOUNT_POINT + "/OnBase/rm_DVVendorContacts.parquet"

#loads
dfZoomData = spark.read.load(ZoomDataLoad)
rm_DVStatementRequestActivityRecords = spark.read.load(rm_DVStatementRequestActivityRecords).withColumn("add_minutes", col("CreatedDate") + expr("INTERVAL 5 minutes")).withColumn("sub_minutes", col("CreatedDate") - expr("INTERVAL 5 minutes"))
rm_DVStatementRequests = spark.read.load(rm_DVStatementRequests)
rm_DVVendorContacts = spark.read.load(rm_DVVendorContacts)

#cleanup
rm_DVStatementRequestActivityRecords = rm_DVStatementRequestActivityRecords.withColumn("ActivityUser", trim(col("ActivityUser")))
rm_DVVendorContacts = rm_DVVendorContacts.withColumn("Phone", regexp_replace(col("Phone"), r'\D', ''))
dfZoomData = dfZoomData.withColumn("Callee_number", regexp_replace(col("Callee_number"), r'\D', '')).withColumn("Callee_number", regexp_extract(col("Callee_number"),'\d{10}$',0)).withColumn("Caller_number", regexp_replace(col("Caller_number"), r'\D', '')).withColumn("Caller_number", regexp_extract(col("Caller_number"),'\d{10}$',0))

#temp tables
dfZoomData.createOrReplaceTempView("ZoomData")
rm_DVStatementRequestActivityRecords.createOrReplaceTempView("rm_DVStatementRequestActivityRecords")
rm_DVStatementRequests.createOrReplaceTempView("rm_DVStatementRequests")
rm_DVVendorContacts.createOrReplaceTempView("rm_DVVendorContacts")

In [0]:
%sql
--Select * FROM rm_DVVendorContacts

In [0]:
%sql
DROP TABLE IF EXISTS StatementRequestActivityRecordsCallData

In [0]:
%sql
--CREATE TABLE StatementRequestActivityRecordsCallData
SELECT SRAR.ObjectID
      ,SRAR.CreatedDate
      ,SRAR.ReferenceNumber
      ,SRAR.CustomerVendorName
      ,SRAR.JobNumber
      ,SRAR.JobName
      ,SRAR.ActivityUser
      ,SRAR.ActivityDate
      ,SRAR.FollowUpDate
      ,SRAR.STNID
      ,SRAR.Notes
      ,SRAR.Outcome
      ,SRAR.ActivityType
      ,SRAR.CustVendorObjectID
      ,SRAR.VendorContactObjectID
      ,SRAR.StatementRequestObjectID
      ,SRAR.CreatedBy
      ,SRAR.sub_minutes
      ,SRAR.add_minutes
      ,vc.Phone
      ,vc.ContactType
      ,SR.Status
      ,SR.StatementWNC
      ,ZD.Caller_name
      ,ZD.Caller_number
      ,ZD.Callee_name
      ,ZD.Callee_number
      ,ZD.duration
      ,ZD.result
      ,ZD.direction
      ,ZD.type
      ,ZD.call_id
      ,ZD.id
      ,ZD.path
      ,ZD.call_type
      ,ZD.extension_number
      ,ZD.date_time_est
  FROM rm_DVStatementRequestActivityRecords AS SRAR
  INNER JOIN rm_DVVendorContacts AS VC ON SRAR.VendorContactObjectID = VC.ObjectID
  INNER JOIN rm_DVStatementRequests AS SR ON SRAR.StatementRequestObjectID = SR.ObjectID
  INNER JOIN ZoomData AS ZD ON 
  
  --ZD.direction = "outbound"
  --AND UPPER(SRARC.ActivityUser) = UPPER(ZD.Caller_name) 
  AND VC.Phone LIKE ZD.Callee_number + '%'
  --AND SRAR.ActivityType = "Called Vendor"
  AND (ZD.date_time_est BETWEEN SRAR.sub_minutes AND SRAR.add_minutes)
  --OR
  --(ZD.direction = "inbound"
  --AND UPPER(SRAR.ActivityUser) = UPPER(ZD.Callee_name) 
  --AND VC.Phone LIKE (ZD.Caller_number+'%')
  --AND SRAR.ActivityType = 'Received Call / Email' 
  --AND (ZD.date_time_est BETWEEN SRAR.sub_minutes AND SRAR.add_minutes))
  ORDER BY SRAR.CreatedDate DESC
  

ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ActivityUser,ActivityDate,FollowUpDate,STNID,Notes,Outcome,ActivityType,CustVendorObjectID,VendorContactObjectID,StatementRequestObjectID,CreatedBy,sub_minutes,add_minutes,Phone,ContactType,Status,StatementWNC,Caller_name,Caller_number,Callee_name,Callee_number,duration,result,direction,type,call_id,id,path,call_type,extension_number,date_time_est


In [0]:
%sql
--SELECT * FROM StatementRequestActivityRecordsCallData 